# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 8:50AM,237936,15,8052296,"Carwin Pharmaceutical Associates, LLC",Released
1,May 24 2022 8:50AM,237936,15,8053736,"Carwin Pharmaceutical Associates, LLC",Released
2,May 24 2022 8:47AM,237937,18,IF-2089958,"Blinc International, Inc.",Released
3,May 24 2022 8:47AM,237935,15,VT80172698,"Virtus Pharmaceuticals, LLC",Released
4,May 24 2022 8:47AM,237935,15,VT80172699,"Virtus Pharmaceuticals, LLC",Released
5,May 24 2022 8:47AM,237935,15,VT80172700,"Virtus Pharmaceuticals, LLC",Released
6,May 24 2022 8:47AM,237935,15,VT80172701,"Virtus Pharmaceuticals, LLC",Released
7,May 24 2022 8:47AM,237935,15,VT80172703,"Virtus Pharmaceuticals, LLC",Released
8,May 24 2022 8:47AM,237935,15,VT80172704,"Virtus Pharmaceuticals, LLC",Released
9,May 24 2022 8:47AM,237935,15,VT80172705,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
40,237934,Released,1
41,237935,Released,12
42,237936,Released,2
43,237937,Released,1
44,237940,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237934,NaN,NaN,1.0
237935,NaN,NaN,12.0
237936,NaN,NaN,2.0
237937,NaN,NaN,1.0
237940,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237808,0.0,0.0,1.0
237813,0.0,0.0,3.0
237815,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237808,0,0,1
237813,0,0,3
237815,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237808,0,0,1
3,237813,0,0,3
4,237815,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237808,,,1
3,237813,,,3
4,237815,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC"
2,May 24 2022 8:47AM,237937,18,"Blinc International, Inc."
3,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC"
15,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated"
16,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC"
30,May 24 2022 8:35AM,237929,15,"Brookfield Pharmaceuticals, LLC"
70,May 24 2022 8:33AM,237928,15,"Alliance Pharma, Inc."
91,May 24 2022 8:30AM,237934,19,ACG North America LLC
92,May 24 2022 8:28AM,237926,18,Hush Hush
112,May 24 2022 8:27AM,237932,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",,,2
1,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",,,1
2,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",,,12
3,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",,,1
4,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",,,14
5,May 24 2022 8:35AM,237929,15,"Brookfield Pharmaceuticals, LLC",,,40
6,May 24 2022 8:33AM,237928,15,"Alliance Pharma, Inc.",,,21
7,May 24 2022 8:30AM,237934,19,ACG North America LLC,,,1
8,May 24 2022 8:28AM,237926,18,Hush Hush,,,20
9,May 24 2022 8:27AM,237932,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",2,,
1,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",1,,
2,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",12,,
3,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",1,,
4,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",14,,
5,May 24 2022 8:35AM,237929,15,"Brookfield Pharmaceuticals, LLC",40,,
6,May 24 2022 8:33AM,237928,15,"Alliance Pharma, Inc.",21,,
7,May 24 2022 8:30AM,237934,19,ACG North America LLC,1,,
8,May 24 2022 8:28AM,237926,18,Hush Hush,20,,
9,May 24 2022 8:27AM,237932,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",2,,
1,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",1,,
2,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",12,,
3,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",1,,
4,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",14,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",2.0,NaN,NaN
1,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",1.0,NaN,NaN
2,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",12.0,NaN,NaN
3,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",1.0,NaN,NaN
4,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",14.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 8:50AM,237936,15,"Carwin Pharmaceutical Associates, LLC",2.0,0.0,0.0
1,May 24 2022 8:47AM,237937,18,"Blinc International, Inc.",1.0,0.0,0.0
2,May 24 2022 8:47AM,237935,15,"Virtus Pharmaceuticals, LLC",12.0,0.0,0.0
3,May 24 2022 8:44AM,237940,10,"Senseonics, Incorporated",1.0,0.0,0.0
4,May 24 2022 8:42AM,237931,15,"Laser Pharmaceuticals, LLC",14.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,951545,8.0,0.0,0.0
12,475723,1.0,1.0,0.0
15,1189659,89.0,0.0,0.0
16,713702,10.0,2.0,0.0
17,237924,1.0,0.0,0.0
18,1664685,26.0,0.0,0.0
19,1665366,11.0,4.0,17.0
20,1665248,100.0,9.0,0.0
21,475791,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,951545,8.0,0.0,0.0
1,12,475723,1.0,1.0,0.0
2,15,1189659,89.0,0.0,0.0
3,16,713702,10.0,2.0,0.0
4,17,237924,1.0,0.0,0.0
5,18,1664685,26.0,0.0,0.0
6,19,1665366,11.0,4.0,17.0
7,20,1665248,100.0,9.0,0.0
8,21,475791,2.0,0.0,0.0
9,22,475773,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,0.0,0.0
1,12,1.0,1.0,0.0
2,15,89.0,0.0,0.0
3,16,10.0,2.0,0.0
4,17,1.0,0.0,0.0
5,18,26.0,0.0,0.0
6,19,11.0,4.0,17.0
7,20,100.0,9.0,0.0
8,21,2.0,0.0,0.0
9,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,12,Released,1.0
2,15,Released,89.0
3,16,Released,10.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
Executing,0.0,1.0,0.0,2.0,0.0,0.0,4.0,9.0,0.0,2.0
Released,8.0,1.0,89.0,10.0,1.0,26.0,11.0,100.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,2.0,0.0,0.0,4.0,9.0,0.0,2.0
2,Released,8.0,1.0,89.0,10.0,1.0,26.0,11.0,100.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,2.0,0.0,0.0,4.0,9.0,0.0,2.0
2,Released,8.0,1.0,89.0,10.0,1.0,26.0,11.0,100.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()